In [1]:
import os
import sys
import random
import json
import importlib
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from datetime import datetime

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config 
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log
import mrcnn_code.egg as egg

from mrcnn_code.egg import EggDataset, color_splash

importlib.reload(modellib)

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "models")
MODEL_PATH = os.path.join(MODEL_DIR, ".weights.h5")

2025-06-18 08:36:50.516063: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-18 08:36:50.525311: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750228610.536240    1464 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750228610.539393    1464 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750228610.548361    1464 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

ERROR:tensorflow:An interactive session is already active. This can cause out-of-memory errors or some other unexpected errors (due to the unpredictable timing of garbage collection) in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s). Please use `tf.Session()` if you intend to productionize.


I0000 00:00:1750228612.530289    1464 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6924 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:01:00.0, compute capability: 8.6
I0000 00:00:1750228612.539205    1464 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6924 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:01:00.0, compute capability: 8.6


In [2]:
EGG_DIR = os.path.join(ROOT_DIR, "Data/egg_annot")
config = egg.EggConfig()
model = modellib.MaskRCNN(mode="training", config=config, model_dir=MODEL_DIR)
model.load_weights(MODEL_PATH, by_name=True)
dataset_train = EggDataset()
dataset_train.load_egg(EGG_DIR, "train")
dataset_train.prepare()

dataset_val = EggDataset()
dataset_val.load_egg(EGG_DIR, "val")
dataset_val.prepare()

I0000 00:00:1750228614.072696    1464 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6924 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:01:00.0, compute capability: 8.6


In [3]:
model.train(dataset_train,
            dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=20,
            layers='heads')


Starting at epoch 0. LR=0.001

Checkpoint Path: /home/tibor/Documents/Python/mosquito-egg-identification/models/egg20250618T0836/.weights.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
rpn_model              (Functional)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4         (TimeDistributed)
mrcnn_bbox_fc          (TimeDistributed)
mrcnn_m

ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.ops`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```


In [11]:
# Zuerst im Inferenz-Modus erstellen
inference_config = egg.EggConfig()
inference_config.NAME = "egg_inference"
model = modellib.MaskRCNN(mode="inference", 
                         config=inference_config,
                         model_dir=MODEL_DIR)

# Dann auf Training umstellen
model.keras_model.trainable = True
model.set_trainable(layer_regex="all")

Selecting layers to train


In [12]:
import h5py
import numpy as np
from mrcnn.model import ProposalLayer, PyramidROIAlign  # Import der Custom Layers

COCO_MODEL_PATH = os.path.join(MODEL_DIR, "mask_rcnn_coco.h5")

def load_coco_weights_with_mismatch(model, coco_path):
    # Layer, die trotz Shape-Unterschied übertragen werden können
    TRANSFERABLE_LAYERS = [
        'mrcnn_class_conv1', 'mrcnn_class_bn1',
        'mrcnn_class_conv2', 'mrcnn_class_bn2',
        'mrcnn_mask_conv1', 'mrcnn_mask_bn1',
        'mrcnn_mask_conv2', 'mrcnn_mask_bn2',
        'mrcnn_mask_conv3', 'mrcnn_mask_bn3',
        'mrcnn_mask_conv4', 'mrcnn_mask_bn4'
    ]
    
    with h5py.File(coco_path, 'r') as f:
        layer_names = [n.decode('utf8') for n in f.attrs['layer_names']]
        
        for layer in model.keras_model.layers:
            if layer.name in layer_names:
                try:
                    g = f[layer.name]
                    weight_names = [n.decode('utf8') for n in g.attrs['weight_names']]
                    weights = [np.array(g[wn]) for wn in weight_names]
                    
                    # Spezialbehandlung für bestimmte Layer
                    if layer.name in TRANSFERABLE_LAYERS:
                        print(f"✓ {layer.name} (transferred despite shape difference)")
                        layer.set_weights(weights)
                    elif layer.name == 'mrcnn_class_logits':
                        # Nur die ersten N Gewichte übertragen
                        new_weights = [w[:model.config.NUM_CLASSES] if w.ndim > 0 else w for w in weights]
                        layer.set_weights(new_weights)
                        print(f"✓ {layer.name} (partial transfer)")
                    else:
                        if [w.shape for w in weights] == [w.shape for w in layer.get_weights()]:
                            layer.set_weights(weights)
                            print(f"✓ {layer.name}")
                        else:
                            print(f"✗ {layer.name} (shape mismatch)")
                except Exception as e:
                    print(f"⚠ {layer.name} error: {str(e)}")
            else:
                print(f"☐ {layer.name} (not in COCO)")

# Verwendung:
load_coco_weights_with_mismatch(model, COCO_MODEL_PATH)

✓ input_image
✓ zero_padding2d_1
✓ conv1
✓ bn_conv1
✓ activation_74
✓ max_pooling2d_1
✓ res2a_branch2a
✓ bn2a_branch2a
☐ activation_75 (not in COCO)
✓ res2a_branch2b
✓ bn2a_branch2b
☐ activation_76 (not in COCO)
✓ res2a_branch2c
✓ res2a_branch1
✓ bn2a_branch2c
✓ bn2a_branch1
✓ add_33
✓ res2a_out
✓ res2b_branch2a
✓ bn2b_branch2a
☐ activation_77 (not in COCO)
✓ res2b_branch2b
✓ bn2b_branch2b
☐ activation_78 (not in COCO)
✓ res2b_branch2c
✓ bn2b_branch2c
☐ add_34 (not in COCO)
✓ res2b_out
✓ res2c_branch2a
✓ bn2c_branch2a
☐ activation_79 (not in COCO)
✓ res2c_branch2b
✓ bn2c_branch2b
☐ activation_80 (not in COCO)
✓ res2c_branch2c
✓ bn2c_branch2c
☐ add_35 (not in COCO)
✓ res2c_out
✓ res3a_branch2a
✓ bn3a_branch2a
☐ activation_81 (not in COCO)
✓ res3a_branch2b
✓ bn3a_branch2b
☐ activation_82 (not in COCO)
✓ res3a_branch2c
✓ res3a_branch1
✓ bn3a_branch2c
✓ bn3a_branch1
☐ add_36 (not in COCO)
✓ res3a_out
✓ res3b_branch2a
✓ bn3b_branch2a
☐ activation_83 (not in COCO)
✓ res3b_branch2b
✓ bn3b_bra

In [23]:
model.keras_model.save('model.keras')

In [25]:
model.keras_model.save_weights('.weights.h5')

In [20]:
layers_to_train = [
    'mrcnn_class_logits',
    'mrcnn_bbox_fc',
    'mrcnn_mask',
    'rpn_class',
    'rpn_bbox'
]

# Für jeden Layer einzeln
for layer_pattern in layers_to_train:
    model.set_trainable(layer_pattern)

Selecting layers to train
mrcnn_class_logits     (TimeDistributed)
Selecting layers to train
mrcnn_bbox_fc          (TimeDistributed)
Selecting layers to train
mrcnn_mask             (TimeDistributed)
Selecting layers to train
Selecting layers to train
